In [19]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc read-aloud"><h2>--- Day 16: Dragon Checksum ---</h2><p>You're done scanning this part of the network, but you've left traces of your presence. You need to <span title="If I ever find one of my disks overwritten with a dragon curve, I'll know it was you.">overwrite some disks</span> with random-looking data to cover your tracks and update the local security system with a new checksum for those disks.</p>
<p>For the data to not be suspicious, it needs to have certain properties; purely random data will be detected as tampering. To generate appropriate random data, you'll need to use a modified <a href="https://en.wikipedia.org/wiki/Dragon_curve">dragon curve</a>.</p>
<p>Start with an appropriate initial state (your puzzle input). Then, so long as you don't have enough data yet to fill the disk, repeat the following steps:</p>
<ul>
<li>Call the data you have at this point "a".</li>
<li>Make a copy of "a"; call this copy "b".</li>
<li>Reverse the order of the characters in "b".</li>
<li>In "b", replace all instances of <code>0</code> with <code>1</code> and all <code>1</code>s with <code>0</code>.</li>
<li>The resulting data is "a", then a single <code>0</code>, then "b".</li>
</ul>
<p>For example, after a single step of this process,</p>
<ul>
<li><code>1</code> becomes <code>100</code>.</li>
<li><code>0</code> becomes <code>001</code>.</li>
<li><code>11111</code> becomes <code>11111000000</code>.</li>
<li><code>111100001010</code> becomes <code>1111000010100101011110000</code>.</li>
</ul>
<p>Repeat these steps until you have enough data to fill the desired disk.</p>
<p>Once the data has been generated, you also need to create a checksum of that data. Calculate the checksum <em>only</em> for the data that fits on the disk, even if you generated more data than that in the previous step.</p>
<p>The checksum for some given data is created by considering each non-overlapping <em>pair</em> of characters in the input data.  If the two characters match (<code>00</code> or <code>11</code>), the next checksum character is a <code>1</code>.  If the characters do not match (<code>01</code> or <code>10</code>), the next checksum character is a <code>0</code>. This should produce a new string which is exactly half as long as the original. If the length of the checksum is <em>even</em>, repeat the process until you end up with a checksum with an <em>odd</em> length.</p>
<p>For example, suppose we want to fill a disk of length <code>12</code>, and when we finally generate a string of at least length <code>12</code>, the first <code>12</code> characters are <code>110010110100</code>. To generate its checksum:</p>
<ul>
<li>Consider each pair: <code>11</code>, <code>00</code>, <code>10</code>, <code>11</code>, <code>01</code>, <code>00</code>.</li>
<li>These are same, same, different, same, different, same, producing <code>110101</code>.</li>
<li>The resulting string has length <code>6</code>, which is <em>even</em>, so we repeat the process.</li>
<li>The pairs are <code>11</code> (same), <code>01</code> (different), <code>01</code> (different).</li>
<li>This produces the checksum <code>100</code>, which has an <em>odd</em> length, so we stop.</li>
</ul>
<p>Therefore, the checksum for <code>110010110100</code> is <code>100</code>.</p>
<p>Combining all of these steps together, suppose you want to fill a disk of length <code>20</code> using an initial state of <code>10000</code>:</p>
<ul>
<li>Because <code>10000</code> is too short, we first use the modified dragon curve to make it longer.</li>
<li>After one round, it becomes <code>10000011110</code> (<code>11</code> characters), still too short.</li>
<li>After two rounds, it becomes <code>10000011110010000111110</code> (<code>23</code> characters), which is enough.</li>
<li>Since we only need <code>20</code>, but we have <code>23</code>, we get rid of all but the first <code>20</code> characters: <code>10000011110010000111</code>.</li>
<li>Next, we start calculating the checksum; after one round, we have <code>0111110101</code>, which <code>10</code> characters long (<em>even</em>), so we continue.</li>
<li>After two rounds, we have <code>01100</code>, which is <code>5</code> characters long (<em>odd</em>), so we are done.</li>
</ul>
<p>In this example, the correct checksum would therefore be <code>01100</code>.</p>
<p>The first disk you have to fill has length <code>272</code>. Using the initial state in your puzzle input, <em>what is the correct checksum</em>?</p>
</article>


In [20]:
def convert(a: str) -> str:
    return a + "0" + "".join("1" if l == "0" else "0" for l in reversed(a))


tests_convert = [
    {
        "name": "Example 1",
        "a": "1",
        "expected": "100",
    },
    {
        "name": "Example 2",
        "a": "0",
        "expected": "001",
    },
    {
        "name": "Example 3",
        "a": "11111",
        "expected": "11111000000",
    },
    {
        "name": "Example 4",
        "a": "111100001010",
        "expected": "1111000010100101011110000",
    },
]

run_tests_params(convert, tests_convert)


Test Example 1 passed, for convert.
Test Example 2 passed, for convert.
Test Example 3 passed, for convert.
Test Example 4 passed, for convert.
Success


In [21]:
from itertools import batched

translate = dict(zip(batched("11001001", n=2), "1100"))


def checksum(a: str) -> str:
    return "".join(translate[t] for t in batched(a, n=2))


tests_checksum = [
    {
        "name": "Example 1",
        "a": "11",
        "expected": "1",
    },
    {
        "name": "Example 2",
        "a": "00",
        "expected": "1",
    },
    {
        "name": "Example 3",
        "a": "10",
        "expected": "0",
    },
    {
        "name": "Example 4",
        "a": "01",
        "expected": "0",
    },
    {
        "name": "Example 5",
        "a": "110010110100",
        "expected": "110101",
    },
    {
        "name": "Example 6",
        "a": "110101",
        "expected": "100",
    },
]

run_tests_params(checksum, tests_checksum)


Test Example 1 passed, for checksum.
Test Example 2 passed, for checksum.
Test Example 3 passed, for checksum.
Test Example 4 passed, for checksum.
Test Example 5 passed, for checksum.
Test Example 6 passed, for checksum.
Success


In [24]:
def dragon_curve(a: str, length: int) -> str:
    while len(a) < length:
        a = convert(a)

    a = checksum(a[:length])
    while len(a) & 1 == 0:
        a = checksum(a[:length])

    return a


tests_dragon_curve = [
    {
        "name": "Example 1",
        "a": "110010110100",
        "length": 12,
        "expected": "100",
    },
    {
        "name": "Example 2",
        "a": "10000",
        "length": 20,
        "expected": "01100",
    },
]

run_tests_params(dragon_curve, tests_dragon_curve)


Test Example 1 passed, for dragon_curve.
Test Example 2 passed, for dragon_curve.
Success


In [25]:
dragon_curve("11101000110010100", 272)

'10100101010101101'

<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>10100101010101101</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>The second disk you have to fill has length <code>35651584</code>. Again using the initial state in your puzzle input, <em>what is the correct checksum</em> for this disk?</p>
</article>

</main>


In [26]:
dragon_curve("11101000110010100", 35651584)

'01100001101101001'

<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>01100001101101001</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>
<p>At this point, you should <a href="/2016">return to your Advent calendar</a> and try another puzzle.</p>
<p>Your puzzle input was <code class="puzzle-input">11101000110010100</code>.</p>

</main>
